In [2]:
import sys
import os
import dtale
import dtale.app as dtale_app
dtale_app.JUPYTER_SERVER_PROXY = True
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import canalyst_candas.candas as cd
import canalyst_candas.candas_datareader as cdr
from canalyst_candas.configuration.config import Config
config_info = {
  "canalyst_api_key": "",
  "s3_access_key_id": "",
  "s3_secret_key": "",
  "fred_key": "",
  "default_dir": "",
  "mds_host": "",
  "wp_host": ""
}
config = Config(config=config_info)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

## JPM Read Through for credit and debit card volumes to V and MA

### First we search for common time series among JPM V and MA with 'credit card sales volume debit payments US'

In [ ]:
canalyst_search = cd.Search(config)
df = canalyst_search.search_time_series(ticker=['JPM US','V US','MA US'],
                                        unit_type='currency',
                     time_series_description='credit card sales volume debit payments volume US')[['ticker','time_series_description','time_series_name','search_matches']]
df= df[df['search_matches']>3]

Building search index...


In [ ]:
df

### Now that we have our series, let's pull the data from Canalyst

In [ ]:
ticker = "JPM US"

In [ ]:
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)

In [ ]:
df_jpm_credit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      is_historical=True,
                      time_series_name='z_3FRM7X0148_MO_OS_Creditcardsalesvolume')[['ticker','time_series_name','period_end_date','value']]

df_jpm_debit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      is_historical=True,
                      time_series_name='z_3FRM7X0148_MO_OS_Debitcardsalesvolume')[['ticker','time_series_name','period_end_date','value']]
 


### Next we get Visa data from Canalyst

In [ ]:
ticker = "V US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_visa_credit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_2CRCA60110_MO_OS_usconsumercreditpaymentsvolume')[['ticker','time_series_name','period_end_date','value']]

df_visa_debit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_2CRCA60110_MO_OS_usconsumerdebitpaymentsvolume')[['ticker','time_series_name','period_end_date','value']]


### We need to do a deeper search on Mastercard time series, add 'programs' and 'unitedstates' to our search

In [ ]:
#add 'programs' and 'unitedstates' to our search
df = canalyst_search.search_time_series(ticker=['MA US'],
                                        unit_type='currency',
                     time_series_description='charge debit programs volume UnitedStates')[['ticker','time_series_description','time_series_name','search_matches']]
df= df[df['search_matches']>2]
df

### Given the better time series matches, same data pull for MA

In [ ]:
ticker = "MA US"
model_set = cd.ModelSet(ticker_list=[ticker], 
                            config=config)
df_mastercard_credit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_GSYQ3D0168_MO_OS_UnitedStatescreditandchargeprogramspurchasevolume_3')[['ticker','time_series_name','period_end_date','value']]
df_mastercard_debit_volume = model_set.model_frame(ticker=ticker,
                      period_duration_type='fiscal_quarter',
                      time_series_name='z_GSYQ3D0168_MO_OS_UnitedStatesdebitprogramspurchasevolume_3')[['ticker','time_series_name','period_end_date','value']]


### Now we use the regress_dataframe function from Candas DataReader
#### R squared for JPM vs V and MA, first by credit then by debit in turn

In [ ]:
df = pd.merge(df_jpm_credit_volume,df_visa_credit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

#Visa fit is not as good because:
#for Visa, "Service revenues in a given quarter are assessed based on nominal payments volume in the prior quarter."
#footnote 1 p 37 Visa annual report

In [ ]:
df = pd.merge(df_jpm_credit_volume,df_mastercard_credit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

In [ ]:
df = pd.merge(df_jpm_debit_volume,df_visa_debit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]

In [ ]:
df = pd.merge(df_jpm_debit_volume,df_mastercard_debit_volume,how='inner',left_on='period_end_date',right_on='period_end_date')[['period_end_date','value_x','value_y']]
cdr.regress_dataframe(df,'value_y','value_x',y_filter="").tables[0]